# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otocznia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3.Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import loadmat
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/'

fileNames = ["MR_data.mat"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

#TODO Samodzielna



In [ ]:
from scipy.io import loadmat
mat = loadmat('MR_data.mat')

In [ ]:
I1 = mat['I_noisy1']
I2 = mat['I_noisy2']
I3 = mat['I_noisy3']
I4 = mat['I_noisy4']
I5 = mat['I_noisefree']

In [ ]:
fig,axs = plt.subplots(1,5)
fig.set_size_inches(20, 10)
axs[0].imshow(I1)
axs[0].axis('off')
axs[1].imshow(I2)
axs[1].axis('off')
axs[2].imshow(I3)
axs[2].axis('off')
axs[3].imshow(I4)
axs[3].axis('off')
axs[4].imshow(I5)
axs[4].axis('off')

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()
    

In [ ]:
mask = fgaussian(3,0.9)
mesh(mask,3)

In [ ]:
def classic(window,x,sigma):
  X,Y = window.shape
  mask = fgaussian(x,sigma)
  
  s = 0

  for i in range(X):
    for j in range(Y):
      s += window[i,j]*mask[i,j]
  return (1/ mask.sum()) *s



def fun(I,paddingX,paddingY):
  I1c = I.copy()
  X,Y = I1c.shape
  dst = I1c.copy()
  for i in range(paddingX,X-paddingX):
    for j in range(paddingY,Y-paddingY):
      ctx = I1c[i-(paddingX):i+(paddingX)+1,j-(paddingY):j+(paddingY)+1]
      dst[i,j] = classic(ctx,paddingX*2 +1,1)

  return dst


In [ ]:
dst1 = fun(I1,1,1)
dst2 = fun(I2,1,1)
dst3 = fun(I3,1,1)
dst4 = fun(I4,1,1)
dst5 = fun(I5,1,1)

In [ ]:
fig, axs = plt.subplots(5, 2)
fig.set_size_inches(20, 30)
axs[0,0].imshow(I1,'gray')
axs[0,0].axis('off')
axs[0,1].imshow(dst1,'gray')
axs[0,1].axis('off')
axs[1,0].imshow(I2,'gray')
axs[1,0].axis('off')
axs[1,1].imshow(dst2,'gray')
axs[1,1].axis('off')
axs[2,0].imshow(I3,'gray')
axs[2,0].axis('off')
axs[2,1].imshow(dst3,'gray')
axs[2,1].axis('off')
axs[3,0].imshow(I4,'gray')
axs[3,0].axis('off')
axs[3,1].imshow(dst4,'gray')
axs[3,1].axis('off')
axs[4,0].imshow(I5,'gray')
axs[4,0].axis('off')
axs[4,1].imshow(dst5,'gray')
axs[4,1].axis('off')

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gausowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def wariance(war,window):
  p = window[0,0]
  x = window[window.shape[0]-1,window.shape[1]-1]
  n = (p-x)*(p-x)
  m = 2*war*war
  l = np.exp(-(n)/(m))
  return l
  
def classicv2(window,x,sigma,var):
  X,Y = window.shape
  mask = fgaussian(x,sigma)
  center = window[X // 2, Y // 2]
  
  s = 0
  G = out = 0


  for i in range(X):
    for j in range(Y):
      p = window[0,0]
      x = window[window.shape[0]-1,window.shape[1]-1]
      n = (p-x)*(p-x)
      m = 2*var*var
      gamma = np.exp(-(n)/(m))
      G += mask[i, j] * gamma
      out += mask[i, j] * gamma * window[i, j]
      s += window[i,j]*mask[i,j]
  return 1 / G * out



def fun2(I,paddingX,paddingY,war):
  I1c = I.copy()
  X,Y = I1c.shape
  dst = I1c.copy()
  for i in range(paddingX,X-paddingX):
    for j in range(paddingY,Y-paddingY):
      ctx = I1c[i-(paddingX):i+(paddingX)+1,j-(paddingY):j+(paddingY)+1]
      #dst[i,j] = classic(ctx,paddingX*2 +1,1) * wariance(war,ctx)
      dst[i,j] = classicv2(ctx,paddingX*2+1,1,war)

  return dst

In [ ]:
dst1 = fun2(I1,2,2,250)
dst2 = fun2(I2,2,2,250)
dst3 = fun2(I3,2,2,1000)
dst4 = fun2(I4,2,2,1000)
dst5 = fun2(I5,2,2,250)


In [ ]:
fig, axs = plt.subplots(5, 2)
fig.set_size_inches(20, 30)
axs[0,0].imshow(I1,'gray')
axs[0,0].axis('off')
axs[0,1].imshow(dst1,'gray')
axs[0,1].axis('off')
axs[1,0].imshow(I2,'gray')
axs[1,0].axis('off')
axs[1,1].imshow(dst2,'gray')
axs[1,1].axis('off')
axs[2,0].imshow(I3,'gray')
axs[2,0].axis('off')
axs[2,1].imshow(dst3,'gray')
axs[2,1].axis('off')
axs[3,0].imshow(I4,'gray')
axs[3,0].axis('off')
axs[3,1].imshow(dst4,'gray')
axs[3,1].axis('off')
axs[4,0].imshow(I5,'gray')
axs[4,0].axis('off')
axs[4,1].imshow(dst5,'gray')
axs[4,1].axis('off')